In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df_train= pd.read_csv('Arabic Sentiment Analysis Dataset - SS2030.csv')
df_train.head()

,text,Sentiment
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,1
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,1
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,1
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,1


In [2]:
#number of positive and negative reviews
df_train['Sentiment'].value_counts()

Sentiment
1    2436
0    1816
Name: count, dtype: int64

In [3]:
import aranorm
df_train['text']=df_train['text'].apply(aranorm.normalize_tweet)
df_train

,text,Sentiment
0,حقوق المراه يوجدرابط,1
1,حقوق المراه في الاسلام يوجدرابط,1
2,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,1
3,حقوق المراه التي تضمنها لها وزاره العدل يوجدرابط,1
4,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,1
...,...,...
4247,غرد بحبك لمحمد بن سلمان,1
4248,غرد بحبك لمحمد بن سلمان محمدبن سلمان احبه الله...,1
4249,غرد بحبك لمحمد بن سلمان الله يحفظك يا ذخر الوط...,1
4250,غرد بحبك لمحمد بن سلمان الله يحفظه ويحميه ويقو...,1


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [5]:
#over sampling
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


# Load your dataset (assuming you have it in a DataFrame format)
# Replace the following line with code to load your dataset
# df_train = pd.read_csv('your_dataset.csv')
# df_train.head()

# Define maximum sequence length and vocabulary size
max_len = 300
vocab_size = 10000

# Define a list of Arabic stopwords outside of the Numba-compiled function
arabic_stopwords = list(set(stopwords.words('arabic')))

# Tokenize Arabic text using NLTK's word_tokenize
# You can further customize this tokenizer as needed
def tokenize_text(text):
    return word_tokenize(text)

df_train['tokenized_text'] = df_train['text'].apply(tokenize_text)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df_train['tokenized_text'])
sequences = tokenizer.texts_to_sequences(df_train['tokenized_text'])

# Pad the sequences
data_padded = pad_sequences(sequences, maxlen=max_len)

# Encode the labels
label_encoder = LabelEncoder()
data_labels = label_encoder.fit_transform(df_train['Sentiment'])
data_labels = to_categorical(data_labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_padded, data_labels, test_size=0.2, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

In [10]:
# Model parameters
embedding_dim = 128
lstm_units = 64

# Build the LSTMmodel1
LSTMmodel1 = Sequential()
LSTMmodel1.add(Embedding(vocab_size, embedding_dim))
LSTMmodel1.add(LSTM(lstm_units))
LSTMmodel1.add(Dense(2, activation='sigmoid'))

# Compile the LSTMmodel1
LSTMmodel1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTMmodel1
history = LSTMmodel1.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluate the LSTMmodel1
loss, accuracy = LSTMmodel1.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.6496 - loss: 0.5933 - val_accuracy: 0.8385 - val_loss: 0.3517
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9364 - loss: 0.1979 - val_accuracy: 0.8678 - val_loss: 0.3233
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9866 - loss: 0.0589 - val_accuracy: 0.8722 - val_loss: 0.4332
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9965 - loss: 0.0138 - val_accuracy: 0.8708 - val_loss: 0.4695
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.9976 - loss: 0.0126 - val_accuracy: 0.8605 - val_loss: 0.6758
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9998 - loss: 0.0032 - val_accuracy: 0.8576 - val_loss: 0.5989
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.8546 - val_loss: 0.7753
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 1.0000 - loss: 7.2075e-04 - val_accuracy: 0.8590